In [1]:
# Loading necessary libraries 

import pandas as pd 
import numpy as np
import dtale
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Legend, Range1d
from bokeh import palettes, layouts

In [3]:
# Helper functions for dataframe manipulation for easier access

goods = [str(i) for i in range(0 , 10)]
services = ['travel', 'transport', 'ict', 'financial']
goods_and_services = goods + services


def dataset_path(digit=1 , year = "all" , partner=False):
# Generates dataset path for the specified file type
    
    digit = digit
    year = year
    partner = partner

    if(partner):
        partner = "partner_"
    else:
        partner = ""
    

    if(digit==1):
        digit = "section_"
    else:
        digit = f"{digit}digit_"


    if(year == "all"):
        year = "year"
    else:
        year = f"year_{year}"

    dataset_path = f"./data_files/country_{partner}sitcproduct{digit}{year}.dta"

    return dataset_path


def single_digitizer(s):
# converts 4 digit SITC product code to its single digit parent code
    
    if(s[0] in goods):
        return s[0]
    else:
        return "services"
    

def double_digitizer(s):
# converts 4 digit SITC product code to its double digit parent code

    if(s[0] in goods):
        return s[0:2]
    else:
        return s


In [38]:
# Loading helper data
product_codes = pd.read_excel("./data_files/sitc_product_codes.xlsx" , sheet_name="SITC2") # Maps SITC product codes to product descriptions
product_codes.drop(columns=["Classification" , "Level" , "IsBasicLevel"] , inplace=True)
product_codes.loc[len(product_codes.index)] = ["services" , "services" , np.nan]

def get_product_description(s):
# Retrieves product description from product codes table
    if(int(s) in product_codes["Code"]):
        return product_codes[product_codes["Code"] == s]["Description"].iloc[0]
    else:
        return np.nan

country_codes = pd.read_excel("./data_files/country_codes.xls") # Maps 3-digit country code to country name
country_codes.drop([0 , 1] , inplace=True)
country_codes.rename(columns={"ISO 3-Digit Alpha Country Code":"code" , "Unnamed: 1":"definition"} , inplace=True)
country_codes.set_index(np.arange(251) , inplace=True)

# Loading and manipulating dataset for gdp
gdp_file_path = "./data_files/gdp_all.xlsx"
gdp_df = pd.read_excel(gdp_file_path)
gdp_df.drop(["Unnamed: 0"] , axis=1 , inplace=True)
gdp_df.rename(columns={"Country Name": "country" , "Country Code":"location_code"} , inplace=True)
gdp_df = pd.melt(gdp_df, id_vars=['country' , "location_code" ], value_vars=[i for i in range(1960 , 2023)] ,var_name='year', value_name='gdp').sort_values(['country' , "year"] , ascending=[True , True])

# Loading raw 4 digit data using helper functions
four_digit_product_data = pd.read_stata(dataset_path(digit=4))

# Basic data manipulation for different use cases
all_digit = four_digit_product_data.rename(columns={"sitc_product_code" : "sitc_product_code_4"})
all_digit["sitc_product_code_1"] = all_digit["sitc_product_code_4"].apply(single_digitizer)
all_digit["sitc_product_code_2"] = all_digit["sitc_product_code_4"].apply(double_digitizer)

# loading the default datset set by dataset_path()
df = pd.read_stata(dataset_path())
# Basic Manipulations and calculations
df = pd.merge(df , product_codes , how="left" , left_on=["sitc_product_code"] , right_on=["Code"])
df = pd.merge(df , country_codes , how = "left" , left_on=["location_code"] , right_on=["code"])
df.rename(columns={"definition":"country"} , inplace=True)
df["trade_balance"] = df["export_value"]-df["import_value"]
df["trade_volume"] = df["export_value"] + df["import_value"]

trade_balance = df.groupby(by = ["location_code" , "year"]).aggregate({"trade_balance" :"sum" , "trade_volume":"sum"})
trade_balance = pd.merge(trade_balance , gdp_df , how="left" , on=["location_code" , "year"])
trade_balance["netex_by_gdp"] = 100*trade_balance["trade_balance"]/trade_balance["gdp"]
trade_balance["trade_by_gdp"] = 100*trade_balance["trade_volume"]/trade_balance["gdp"]

# Loading and manipulating dataset for trade as percentage of gdp
file_path = "./data_files/trade_percentage_of_gdp_all_countries.xlsx"
trade_percentage_gdp = pd.read_excel(file_path , sheet_name="Country-Timeseries")
trade_percentage_gdp.rename(columns={"Country Name": "country"} , inplace=True)
trade_percentage_gdp.drop(["Indicator Name"] , axis=1 , inplace=True)
trade_percentage_gdp = pd.merge(trade_percentage_gdp , gdp_df[["country"  ,"location_code"]] , left_on=["country"] , right_on=["country"])
trade_percentage_gdp = pd.melt(trade_percentage_gdp, id_vars=['country' , "location_code"], value_vars=[str(i) for i in range(1988 , 2022)] ,var_name='year', value_name='trade_percentage').sort_values(['country' , "year"] , ascending=[True , True])

# Attempted to load coordinates data to plot on a map
coords = pd.read_excel("./data_files/latitudes_and_longitudes.xlsx")
r_major = 6378137.000
coords["m_x"] = np.radians(coords["longitude"])*(6378137.000)
np.seterr(divide="ignore")
coords["m_y"] = (180.0/np.pi) * np.log(np.tan((np.pi/4.0) + (coords["latitude"]*(np.pi/360.0)))) * (coords["m_x"]/coords["longitude"])
np.seterr(divide="warn")

# Loading another of the available datasets
country_partner = pd.read_stata(dataset_path(2 , "all" , True))
country_partner.drop(labels=["product_id" , "location_id" , "partner_id" , "sitc_eci" , "sitc_coi"] , axis=1 ,inplace=True)
country_partner.rename({"sitc_product_code":"sitc_product_code_2"} ,axis=1 , inplace=True)
country_partner["sitc_product_code_1"] = country_partner["sitc_product_code_2"].apply(single_digitizer)

In [5]:
# Grouping data to analyse imports and exports products from any (country , year)

imports_exports = df.groupby(["location_code" , "year" , "sitc_product_code"]).aggregate(func = {"export_value":"sum" , "import_value":"sum" , "Description":"sum" , "country":"sum"})
total_imports_exports = df.groupby(["location_code" , "year"]).aggregate(func = {"export_value":"sum" , "import_value":"sum"})
import_export_cols = ["country" , "year" , "sitc_product_code", "export_value" , "export_value_percentage", "import_value" , "import_value_percentage" , "location_code" , "Description" , "global_market_share"]
imports_exports = pd.merge(df , total_imports_exports , how="left" , on=["location_code" , "year"] , suffixes = ["" , "_percentage"])[import_export_cols]
imports_exports["export_value_percentage"] = (100*imports_exports["export_value"])/imports_exports["export_value_percentage"]
imports_exports["import_value_percentage"] = (100*imports_exports["import_value"])/imports_exports["import_value_percentage"]
imports_exports["export_value_percentage"] = imports_exports["export_value_percentage"].fillna(0)
imports_exports["import_value_percentage"] = imports_exports["import_value_percentage"].fillna(0)

In [14]:
# Grouping data to analyse countries that are importers and exporters of a (product , year)

importers_exporters = df.groupby(["sitc_product_code" , "year" , "location_code"]).aggregate(func={"export_value":"sum" , "import_value":"sum"})
total_imports_exports = df.groupby(["sitc_product_code" , "year"]).aggregate(func = {"export_value" : "sum" , "import_value":"sum"})
importers_exporters["location_code"] = importers_exporters.index.get_level_values("location_code")
importers_exporters = pd.merge(importers_exporters , total_imports_exports , how="left" , on=["sitc_product_code" , "year"] , suffixes=["" , "_percentage"])
importers_exporters["export_value_percentage"] = (100*importers_exporters["export_value"])/importers_exporters["export_value_percentage"]
importers_exporters["import_value_percentage"] = (100*importers_exporters["import_value"])/importers_exporters["import_value_percentage"]
exporters = importers_exporters.sort_values(by = ["sitc_product_code" , "year" , "export_value_percentage"] , ascending=[True , True , False])
exporters = exporters[["location_code" , "export_value" , "export_value_percentage" , "import_value" , "import_value_percentage"]]
importers = importers_exporters.sort_values(by = ["sitc_product_code" , "year" , "import_value_percentage"] , ascending=[True , True , False])
importers = importers[["location_code" , "export_value" , "export_value_percentage" , "import_value" , "import_value_percentage"]]

In [468]:
# Plot: export value grouped by single digit product code vs year (country)

country = "IND"

products = list((imports_exports[np.isin(imports_exports["sitc_product_code"] , goods+["services"]) * (imports_exports["location_code"] == country)])["sitc_product_code"].unique())
labels = (product_codes[np.isin(product_codes["Code"] , products)])[["Code" , "Description"]]

exporting_history = figure(title=f"{country} Exports (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Value (USD)")

importing_history = figure(title=f"{country} Imports (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Value (USD)")


inferno = palettes.Category20[labels.shape[0]]

g1 = layouts.row(children=[exporting_history , importing_history] , sizing_mode="stretch_width")
g1_db = (imports_exports[imports_exports["location_code"] == country])[(np.isin(imports_exports[imports_exports["location_code"] == country]["sitc_product_code"] , labels["Code"]))]

g1_db1 = g1_db.sort_values( by = [ "year" ] , ascending=[ True ] )
g1_db2 = g1_db.sort_values( by = [ "year" ] , ascending=[ True ] )

r_max = max(np.max(g1_db1["export_value"]) , np.max(g1_db2["import_value"]))

for (i , product) in enumerate(products):

    y1 = g1_db1[g1_db1["sitc_product_code"] == product]["export_value"]
    x1 = g1_db1[g1_db1["sitc_product_code"] == product]["year"]

    exporting_history.circle(x1 , y1 , color=inferno[i] ,  line_width=1 , size = 4 , fill_alpha=0.1)
    exporting_history.line(x1 , y1 , color=inferno[i] , line_width=2 , line_alpha = 0.8 , legend_label=(str(labels[labels["Code"] == product]["Description"].iloc[0])[:45]))

    y2 = g1_db2[g1_db2["sitc_product_code"] == product]["import_value"].sort_values(ascending=True)
    x2 = g1_db2[g1_db2["sitc_product_code"] == product]["year"]

    importing_history.circle(x2 , y2 , color=inferno[i] ,  line_width=1 , size = 4 , fill_alpha=0.1)
    importing_history.line(x2 , y2 , color=inferno[i] , line_width=2 , line_alpha=0.8 , legend_label=(str(labels[labels["Code"] == product]["Description"].iloc[0])[:45]))

exporting_history.legend.location = "top_left"
exporting_history.legend.click_policy = "mute"

importing_history.legend.location = "top_left"
importing_history.legend.click_policy = "mute"
importing_history.y_range = Range1d(-(r_max/10) , r_max)
exporting_history.y_range = importing_history.y_range

exporting_history.x_range = Range1d(1960,2021)
importing_history.x_range = exporting_history.x_range

show(g1)

In [30]:
# Line Plot: Export value breakdown of a sector vs year (country, sector) 

country = "IND"
sector = "services"
sector_description = product_codes[product_codes["Code"] == sector]["Description"].iloc[0]

if(sector == "services"):
    items_of_interest = services
    products = services
    label_list = services
else:
    items_of_interest = [sector+str(i) for i in range(1 , 10)]

    products = list((all_digit[np.isin(all_digit["sitc_product_code_2"] , items_of_interest) * (all_digit["location_code"] == country)])["sitc_product_code_2"].unique())
    labels = (product_codes[np.isin(product_codes["Code"] , items_of_interest)])[["Code" , "Description"]]
    label_list = list(labels["Description"].apply(lambda x : x[:30]))

country_sector = figure(title=f"{country} {sector_description} (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Value (USD)")

inferno = palettes.Category20[len(products)]

g2_db = all_digit[(all_digit["location_code"] == country) * np.isin(all_digit["sitc_product_code_2"] , products)].sort_values(by = ["year" , "export_value"] , ascending=[True , False])

for (i , product) in enumerate(products):

    y = g2_db[g2_db["sitc_product_code_2"] == product].groupby(by = ["year"]).aggregate({"export_value":"sum"})["export_value"]
    x = g2_db[g2_db["sitc_product_code_2"] == product].groupby(by = ["year"]).aggregate({"export_value":"sum"}).index.get_level_values(0)

    country_sector.circle(x , y , color=inferno[i] ,  line_width=1 , size = 4 , fill_alpha=0.1)
    country_sector.line(x , y , color=inferno[i] , line_width=2 , line_alpha = 0.9 , legend_label=label_list[i])

country_sector.legend.location = "top_left"
country_sector.legend.click_policy = "hide"

show(country_sector)

In [453]:
# Pie Plot: 2 digit level Sector decomposition (country, sector and year)

country = "IND"
sector = "5"
year = 2021

sector_name = (product_codes[product_codes["Code"] == sector]["Description"]).iloc[0]
if(sector == "services"):
    subproducts = services
else:
    subproducts = [sector + str(i) for i in range(1,10)]

g3_db = all_digit[np.isin(all_digit["sitc_product_code_2"] , subproducts) * (all_digit["location_code"] == country)]

pi_db = g3_db.groupby(by=["year" , "sitc_product_code_2"]).aggregate({"export_value":"sum" , "import_value":"sum" , "sitc_product_code_1":"max"})
total_7 = all_digit[(all_digit["sitc_product_code_1"] == sector) * (all_digit["location_code"] == country)].groupby(by=["year" , "sitc_product_code_1"]).aggregate({"export_value":"sum" , "import_value":"sum"})
total_7.reset_index(level=["year" , "sitc_product_code_1"] , drop=False , inplace=True)
pi_db.reset_index(level=["year" , "sitc_product_code_2"] , drop=False , inplace=True)
pi_db = pd.merge(pi_db , total_7 , how="left" , on=["year" , "sitc_product_code_1"] , suffixes=["" , "_total"])

x_range = Range1d(-5,5)
y_range = Range1d(-5,5)

inferno = palettes.Category20[len(subproducts)+1]

exports_angles_data = (pi_db[pi_db["year"] == year][["sitc_product_code_2" , "export_value"]].groupby("sitc_product_code_2").sum())
exports_angles_data.sort_values(by="export_value" , ascending=False , inplace=True)
exports_angles_data.reset_index(0 , drop=False , inplace=True)
exports_angles_data["labels"] = exports_angles_data["sitc_product_code_2"].apply(get_product_description)
labels = list(exports_angles_data["labels"].apply(lambda x : x[:30]))
exports_angles_data.loc["other"] = float(total_7[total_7["year"] == year]["export_value"]) - (pi_db[pi_db["year"] == year]["export_value"].sum())
labels.append("other")

exports_angles_data["angles"] = (2*np.pi*(exports_angles_data["export_value"]/exports_angles_data["export_value"].sum())).cumsum()
exports_angles_data["percentage"] = (100*exports_angles_data["export_value"])/exports_angles_data["export_value"].sum()
exports_start = [0] + list(exports_angles_data["angles"][:-1])
exports_end = list(exports_angles_data["angles"])
exports_percentages = list(exports_angles_data["percentage"])
sectors = exports_angles_data.index.get_level_values(0)


imports_angles_data = (pi_db[pi_db["year"] == year][["sitc_product_code_2" , "import_value" , "export_value"]].groupby("sitc_product_code_2").sum())
imports_angles_data.loc["other"] = float(total_7[total_7["year"] == year]["import_value"]) - (pi_db[pi_db["year"] == year]["import_value"].sum())
imports_angles_data.sort_values(by="export_value" , ascending=False , inplace=True)
subproducts_7 = list(imports_angles_data.index.get_level_values(0))
labels.append("other")

imports_angles_data["angles"] = (2*np.pi*(imports_angles_data["import_value"]/(imports_angles_data["import_value"].sum()))).cumsum()
imports_angles_data["percentage"] = (100*imports_angles_data["import_value"])/imports_angles_data["import_value"].sum()
imports_start = [0] + list(imports_angles_data["angles"][:-1])
imports_end = list(imports_angles_data["angles"])
imports_percentages = list(imports_angles_data["percentage"])


colours = inferno

exports_distribution_source = ColumnDataSource(dict(
    start  = exports_start,
    end    = exports_end,
    colors = colours,
    sectors = sectors,
    labels=labels,
    percentage = exports_percentages
))


imports_distribution_source = ColumnDataSource(dict(
    start  = imports_start,
    end    = imports_end,
    colors = colours,
    sectors = sectors,
    labels = labels,
    percentage = imports_percentages
))

tool_tips_e = [
    ("Share:" , "@percentage{0.0}%")
]

tool_tips_i = [
    ("Share:" , "@percentage{0.0}%")
]

exports_pie_chart = figure(x_range=x_range, 
                         y_range=y_range,
                         tooltips= tool_tips_e,
                         title=f"{country}-{sector_name}-Exports {year}")

imports_pie_chart = figure(x_range=x_range,
                         y_range=y_range,
                         tooltips=tool_tips_i,
                         title=f"{country}-{sector_name}-Imports {year}")

pie_row = layouts.row(children=[exports_pie_chart , imports_pie_chart] , sizing_mode="scale_both")

e = exports_pie_chart.annular_wedge(x=0, y=0, inner_radius=3.2, outer_radius=4.5,
                     start_angle="start", end_angle="end",
                     line_color="white", line_width=3, fill_color="colors" , 
                     source=exports_distribution_source,
                     legend_group="labels")

i = imports_pie_chart.annular_wedge(x=0, y=0, inner_radius=3.2, outer_radius=4.5,
                     start_angle="start", end_angle="end",
                     line_color="white", line_width=3, fill_color="colors" , 
                     source=imports_distribution_source,
                     legend_group="labels")

exports_pie_chart.legend.location = "center"
imports_pie_chart.legend.location = "center"

legend = Legend(location="center")


show(pie_row)


/var/folders/c3/rch9_l9x6pbg6mtk7lg7976r0000gp/T/ipykernel_14584/3062672416.py:31: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/c3/rch9_l9x6pbg6mtk7lg7976r0000gp/T/ipykernel_14584/3062672416.py:43: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/c3/rch9_l9x6pbg6mtk7lg7976r0000gp/T/ipykernel_14584/3062672416.py:57: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('colors', 10), ('end', 10), ('labels', 11), ('percentage', 10), ('sectors', 10), ('start', 10)

/var/folders/c3/rch9_l9x6pbg6mtk7lg7976r0000gp/T/ipykernel_14584/3062672416.py:67: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('colors', 10), ('end', 10), ('labels', 11), ('percentage', 10), ('sectors', 10), ('start', 10)



In [18]:
# Line Plot: Trade as a percentage of GDP for countries (countries)

countries = ["IND" , "JPN" , "CHN"]

inferno = palettes.Category20[len(countries)]

trade_percentage = figure(title=f"Trade as percentage of GDP (1988-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Percentage")

for (i , country) in enumerate(countries):

    x = trade_percentage_gdp[trade_percentage_gdp["location_code"] == country]["year"]
    y = trade_percentage_gdp[trade_percentage_gdp["location_code"] == country]["trade_percentage"]

    trade_percentage.circle(x , y , color=inferno[i] ,  line_width=1 , size = 4 , fill_alpha=0.1)
    trade_percentage.line(x , y , color=inferno[i] , line_width=2 , line_alpha=0.8 , legend_label=country)

trade_percentage.legend.location = "top_left"
trade_percentage.legend.click_policy = "mute"

show(trade_percentage)

In [25]:
# Scatter plot: Trade Volume vs GDP for all countries

colour_dictionary = {
    "USA":"blue",
    "DEU":"red",
    "IND":"orange"
}

def colour_code(s):
    if(s in colour_dictionary.keys()):
        return colour_dictionary[s]
    else:
        return "black"
    
trade_balance["location_code"].apply(colour_code)
    

source = ColumnDataSource(dict(
    year = trade_balance["year"],
    country = trade_balance["country"],
    trade_volume = trade_balance["trade_volume"],
    gdp = trade_balance["gdp"],
    trade_balance = trade_balance["trade_balance"],
    colour=trade_balance["location_code"].apply(colour_code)
)
)

tool_tips = [
    ("Country" , "@country"),
    ("Year" , "@year"),
    ("Trade Volume" , "@trade_volume"),
    ("Trade Balance" , "@trade_balance"),
    ("GDP" , "@gdp")
]

trade_volume_vs_gdp = figure(title=f"Trade Volume v/s GDP (1988-2020)" , 
                                  x_axis_label="Trade Volume (USD)" , 
                                  y_axis_label="GDP (USD)",
                                  tooltips=tool_tips)

trade_volume_vs_gdp.scatter(x="trade_volume", y="gdp", size=5, color="colour" ,source=source)
show(trade_volume_vs_gdp)

In [26]:
# Scatter plot : Net Exports vs GDP for all countries

colour_dictionary = {
    "USA":"blue",
    "DEU":"red",
    "IND":"orange",
    "CHN":"pink"
}

def colour_code(s):
    if(s in colour_dictionary.keys()):
        return colour_dictionary[s]
    else:
        return "black"
    
trade_balance["location_code"].apply(colour_code)
    

source = ColumnDataSource(dict(
    year = trade_balance["year"],
    country = trade_balance["country"],
    trade_volume = trade_balance["trade_volume"],
    gdp = trade_balance["gdp"],
    trade_balance = trade_balance["trade_balance"],
    colour=trade_balance["location_code"].apply(colour_code)
)
)

tool_tips = [
    ("Country" , "@country"),
    ("Year" , "@year"),
    ("Trade Volume" , "@trade_volume"),
    ("Trade Balance" , "@trade_balance"),
    ("GDP" , "@gdp")
]

trade_volume_vs_gdp = figure(title=f"GDP v/s Net Exports (1988-2020)" , 
                                  x_axis_label="GDP (USD)" , 
                                  y_axis_label="Trade Balance (USD)",
                                  tooltips=tool_tips)

trade_volume_vs_gdp.scatter(x="gdp", y="trade_balance", size=5, color="colour" ,source=source)
show(trade_volume_vs_gdp)

In [33]:
# Line plot of a few Commonwealth Countries 

common_wealth_countries = ["IND" , "ZAF" , "ISR" , "AUS" , "CAN" , "SGP"]

common_wealth_graph = figure(title=f"Common Wealth Nations GDP - (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Value (USD)")

inferno = palettes.Category20[len(common_wealth_countries)]

g4_db = trade_balance[np.isin(trade_balance["location_code"] , common_wealth_countries)].sort_values(by = ["location_code" , "year" , "gdp"] , ascending=[True , True , False])

for (i , country) in enumerate(common_wealth_countries):

    y = g4_db[g4_db["location_code"] == country].groupby(by = ["year"]).aggregate({"gdp":"sum"})["gdp"]
    x = g4_db[g4_db["location_code"] == country].groupby(by = ["year"]).aggregate({"gdp":"sum"}).index.get_level_values(0)

    common_wealth_graph.circle(x , y , color=inferno[i] ,  line_width=1 , size = 4 , fill_alpha=0.1)
    common_wealth_graph.line(x , y , color=inferno[i] , line_width=2 , line_alpha = 0.9 , legend_label= ((country_codes[country_codes["code"] == common_wealth_countries[i]])["definition"]).iloc[0])

common_wealth_graph.legend.location = "top_left"
common_wealth_graph.legend.click_policy = "hide"

show(common_wealth_graph)

In [59]:
# Attempt to create ain interadtive network graph on the world map

# country_code = "CHN"
# year = 2000

# country_partner.groupby(by=["location_code" , "year" , "partner_code" , "sitc_product_code"]).sum()
# trial = country_partner[(country_partner["location_code"] == country_code) * (country_partner["year"] == year)].sort_values(by = ["export_value"]).groupby(by = ["partner_code"]).head(10)
# trial = country_partner[(country_partner["location_code"] == country_code) * (country_partner["year"] == year)].groupby(by = ["partner_code"]).aggregate({"location_code":"max" , "location_mx":"max" , "location_my":"max" , "export_value":"sum" , "import_value":"sum" , "partner_mx":"max" , "partner_my":"max" , "latitude":"max" , "longitude":"max"})
# trial = country_partner[(country_partner["location_code"] == country_code) * (country_partner["year"] == year)].sort_values(by = ["import_value"] , ascending=[False]).groupby(by = ["partner_code"]).head(10)
# trial.reset_index([0] , drop=False , inplace=True)
# trial["net_exports"] = trial["export_value"]-trial["import_value"]

# coords = pd.read_csv("./data_files/countries_codes_and_coordinates.csv")
# coords.drop(["Alpha-2 code" , "Numeric code"] , axis=1 , inplace=True)
# coords.columns = ["country" , "country_code" , "latitude" , "longitude"]
# cols = ["country_code" , "latitude" , "longitude"]
# for col in cols:
#     coords[col] = coords[col].apply(str.strip)
#     coords[col] = coords[col].apply(lambda x : x[1:-1])
# coords.astype({"latitude":"float" , "longitude":"float"})
# coords.to_excel("./data_files/latitudes_and_longitudes.xlsx")

# from bokeh.tile_providers import Vendors
# palette = palettes.Category20[10]
# data_source = ColumnDataSource(data = trial)
# p = figure(title = 'Avocado Prices by region in the United States', x_axis_type="mercator", y_axis_type="mercator", x_axis_label = 'Longitude', y_axis_label = 'Latitude')
# p.add_tile("CARTODBPOSITRON_RETINA")
# p.circle(x = "partner_mx", y = "partner_my", source=data_source, size=2, fill_alpha = 0.7 , color="blue")
# show(p)

# positions = dict(zip(trial["partner_code"] , zip(trial["latitude"] , trial["latitude"])))
# positions["CHN"] = ( 35 , 105 )

# from bokeh.plotting import from_networkx
# from bokeh.models import Circle, MultiLine, GraphRenderer
# from bokeh.tile_providers import Vendors
# p2 = figure(title = 'Avocado Prices by region in the United States', x_axis_type="mercator", y_axis_type="mercator", x_axis_label = 'Longitude', y_axis_label = 'Latitude')
# p2.add_tile("CARTODBPOSITRON_RETINA")
# g = nx.from_pandas_edgelist(trial, 'location_code', 'partner_code')
# nx.draw(g , positions , p2)
# g = GraphRenderer()
# g.node_renderer.glyph = Circle(size=2, fill_alpha = 0.7 , color="blue")
# g.edge_renderer.data_source.data = dict(x_1=trial["location_mx"] , y_1 = trial["location_my"] , x_2 = trial["partner_mx"] , y_2 = trial["partner_my"])
# network_graph = from_networkx(g, positions, scale=10)
# network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')
# network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
# p2.node_renderer.data_source.data = 
# p2.renderers.append(network_graph)
# show(p2)

# import mplleaflet
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# nx.draw_networkx_nodes(g,pos=positions,node_size=10,node_color='red',alpha=.5)
# nx.draw_networkx_edges(g,pos=positions, alpha=.1)
# nx.draw_networkx_labels(g,positions)
# mplleaflet.display(fig=ax.figure)


# g = nx.from_pandas_edgelist(trial, 'location_code', 'partner_code' , "net_exports")
# mapping = dict((n, i) for i, n in enumerate(g.nodes))
# H = nx.relabel_nodes(g, mapping)

# p = figure(x_range=(-2, 2), y_range=(-2, 2), x_axis_location=None, y_axis_location=None,
#            tools="hover", tooltips="country: @partner_code")
# p.grid.grid_line_color = None

# pos = nx.circular_layout(H , scale=2)
# pos[0] = np.array([0,0])

# nx.draw(H , pos=pos)
# p.renderers.append(graph)

# Add some new columns to the node renderer data source
# graph.node_renderer.data_source.data['index'] = list(range(len(G)))
# graph.node_renderer.data_source.data['colors'] = Category20_20

# graph.node_renderer.glyph.update(size=20, fill_color="colors")

# show(p)


,partner_code,location_code,location_mx,location_my,export_value,import_value,partner_mx,partner_my,latitude,longitude,net_exports
0,ABW,CHN,1.168855e+07,4.163881e+06,6.862494e+06,0.000000e+00,-7.788657e+06,1.402665e+06,12.5000,-69.9667,6.862494e+06
1,AFG,CHN,1.168855e+07,4.163881e+06,2.024301e+07,5.133073e+06,7.235767e+06,3.895304e+06,33.0000,65.0000,1.510994e+07
2,AGO,CHN,1.168855e+07,4.163881e+06,3.380043e+07,1.474154e+09,2.059411e+06,-1.402665e+06,-12.5000,18.5000,-1.440354e+09
3,AIA,CHN,1.168855e+07,4.163881e+06,1.735800e+04,0.000000e+00,-7.031685e+06,2.066831e+06,18.2500,-63.1667,1.735800e+04
4,ALB,CHN,1.168855e+07,4.163881e+06,1.438964e+07,0.000000e+00,2.226390e+06,5.012342e+06,41.0000,20.0000,1.438964e+07
...,...,...,...,...,...,...,...,...,...,...,...
204,WSM,CHN,1.168855e+07,4.163881e+06,1.963410e+06,6.234900e+04,-1.918406e+07,-1.526453e+06,-13.5833,-172.3333,1.901061e+06
205,YEM,CHN,1.168855e+07,4.163881e+06,1.763047e+08,5.996456e+08,5.343336e+06,1.689200e+06,15.0000,48.0000,-4.233408e+08
206,ZAF,CHN,1.168855e+07,4.163881e+06,1.008252e+09,5.332558e+08,2.671668e+06,-3.375646e+06,-29.0000,24.0000,4.749966e+08
207,ZMB,CHN,1.168855e+07,4.163881e+06,3.179647e+07,5.086841e+07,3.339585e+06,-1.689200e+06,-15.0000,30.0000,-1.907194e+07


In [385]:
# Bar chart : growth of Chinese imports in India

def bc_colors(p):
    if(p>25):
        return "#6D3E3E"
    else:
        return "#B0A9A2"

bc1 = country_partner.groupby(by = ["location_code" , "partner_code" , "year"]).aggregate({"export_value":"sum" , "import_value":"sum"}).loc["IND" , "CHN" , 1995:2022]
bc1["import_value_growth"] = bc1['import_value'].pct_change(periods=1) * 100
bc1["import_value_scaled"] = bc1['import_value']/(10**9)
bc1.reset_index(level=[0,1,2] , drop=False , inplace=True)
bc1.sort_values(by="year" , inplace=True)
bc1["year"] = (bc1["year"].astype("int")).astype("str")
bc1["colour"] = bc1["import_value_growth"].apply(bc_colors)

tooltips = [
    ("Growth" , "@import_value_growth{0.00} %"),
    ("Value" , "@import_value_scaled{0.0} billion")
]

bar_chart = figure(x_range = [str(i) for i in list(bc1["year"])] , title = "Growth of India's Imports from China in the 20th Century",
                   toolbar_location="below" , x_axis_label = "Year" , y_axis_label="Percentage" , tooltips=tooltips)


data_source = ColumnDataSource(data=bc1)

bar_chart.vbar(x = "year" , top = "import_value_growth" , source=data_source , width = 0.9 , color="colour")
bar_chart.xaxis.major_label_orientation = 1.2

show(bar_chart)

In [37]:
# Line plot: Share of global exports of countries over time

countries = {"USA" : "#001F3F" , "CHN" : "#C8102E" , "IND":"#FF7F0E" , "DEU":"#333333" , "KOR":"#00468B" , "JPN":"#1A2930"}

def colour_picker(c):
    if c in countries.keys():
        return countries[c]
    else:
        return "black"
    

country_partner['global_export_percentage'] = 100 * country_partner['export_value'] / country_partner.groupby('year')['export_value'].transform('sum')
country_partner['global_import_percentage'] = 100 * country_partner['import_value'] / country_partner.groupby('year')['import_value'].transform('sum')

global_percentages = country_partner.groupby(by = ["location_code" , "year"]).aggregate({"global_export_percentage":"sum" , "global_import_percentage":"sum"})
global_percentages = global_percentages.loc[list(countries.keys())]
global_percentages.reset_index(level=[0,1] , drop=False , inplace=True)
global_percentages["colour"] = global_percentages["location_code"].apply(colour_picker)

global_export_share = figure(title=f"Share of Global Exports (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Value (USD)")

global_import_share = figure(title=f"Share of Global Imports (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Value (USD)")

share_plots = layouts.row(children=[global_export_share , global_import_share] , sizing_mode="stretch_width")
data_source = ColumnDataSource(data=global_percentages)

global_export_share.circle(x="year" , y="global_export_percentage" , color="colour" , source=data_source , line_width=1 , size = 4 , fill_alpha=0.1)
global_import_share.circle(x="year" , y="global_import_percentage" , color="colour" , source=data_source , line_width=1 , size = 4 , fill_alpha=0.1)

for i , country in enumerate(list(countries.keys())):
    global_export_share.line(x="year" , y="global_export_percentage" , color=countries[country] , source=ColumnDataSource(global_percentages[global_percentages["location_code"] == country]) , line_width=2 , line_alpha = 0.8 , legend_label=country)
    global_import_share.line(x="year" , y="global_import_percentage" , color=countries[country] , source=ColumnDataSource(global_percentages[global_percentages["location_code"] == country]) , line_width=2 , line_alpha = 0.8 , legend_label=country)


global_export_share.legend.location = "top"
global_export_share.legend.click_policy = "mute"
global_import_share.legend.click_policy = "mute"

show(share_plots)

In [305]:
# Line Plot: Trade surplus vs year of countries

trade_surplus_countries = {"CHN" : "#D8000C" , "DEU":"#006400" , "KOR":"#1565C0" , "JPN":"#3F51B5"}

def ts_colour(c):
    if c in list(trade_surplus_countries.keys()):
        return trade_surplus_countries[c]
    else:
        return "black"

trade_surplus = country_partner.groupby(by = ["location_code" , "year"]).aggregate({"export_value":"sum" , "import_value":"sum" , "global_export_percentage":"sum" , "global_import_percentage":"sum"})
trade_surplus = trade_surplus.loc[list(trade_surplus_countries.keys())]
trade_surplus["trade_surplus"] = trade_surplus["export_value"]-trade_surplus["import_value"]
trade_surplus.reset_index(level=[0,1] , drop=False , inplace=True)
trade_surplus["colour"] = trade_surplus["location_code"].apply(ts_colour)

trade_surplus_graph = figure(title=f"Trade Surplus for top Exporting Nations (1960-2020)" , 
                                  x_axis_label="Year" , 
                                  y_axis_label="Trade Surplus (USD)")

data_source = ColumnDataSource(data=trade_surplus)

trade_surplus_graph.circle(x="year" , y="trade_surplus" , color="colour" , source=data_source , line_width=1 , size = 4 , fill_alpha=0.1)

for i , country in enumerate(list(trade_surplus_countries.keys())):
    trade_surplus_graph.line(x="year" , y="trade_surplus" , color=trade_surplus_countries[country] , source=ColumnDataSource(trade_surplus[trade_surplus["location_code"] == country]) , line_width=2 , line_alpha = 0.8 , legend_label=country)

trade_surplus_graph.legend.location = "top_left"

show(trade_surplus_graph)

In [40]:
# Pie Chart: Plotting the composition of a sector (country , partner , year)
country = "DEU"
partner = "IND"
year = 2007

pi_db2 = country_partner[(country_partner["partner_code"] == partner) * (country_partner["location_code"] == country) * (country_partner["year"] == year)]
pi_db2['sector_import_percentage'] = 100 * pi_db2['import_value'] / pi_db2.groupby(["sitc_product_code_1"])['import_value'].transform('sum')
pi_db2['sector_export_percentage'] = 100 * pi_db2['export_value'] / pi_db2.groupby(["sitc_product_code_1"])['export_value'].transform('sum')

pi_db2 = pi_db2.groupby(by = ["sitc_product_code_1"]).aggregate({"export_value":"sum" , "import_value":"sum" , "location_code":"max" , "partner_code":"max"})
pi_db2["net_ex"] = pi_db2["export_value"]-pi_db2["import_value"]
pi_db2[["export_value" , "import_value"]]

x_range = Range1d(-5,5)
y_range = Range1d(-5,5)

inferno = palettes.Category20[10]

pi_db2.sort_index(ascending=True , inplace=True)
pi_db2["angles"] = (2*np.pi*(pi_db2["import_value"]/pi_db2["import_value"].sum())).cumsum()
pi_db2["percentage"] = (100*pi_db2["import_value"])/pi_db2["import_value"].sum()
imports_start = [0] + list(pi_db2["angles"][:-1])
imports_end = list(pi_db2["angles"])
percentages = list(pi_db2["percentage"])
sectors = list(pi_db2.index.get_level_values(0))
pi_db2.reset_index(0 , drop=False , inplace=True)
pi_db2["labels"] = pi_db2["sitc_product_code_1"].apply(get_product_description)
labels = list(pi_db2["labels"].apply(lambda x : x[:30]))
colours = inferno

imports_source = ColumnDataSource(dict(
    start  = imports_start,
    end    = imports_end,
    colors = colours,
    sectors = sectors,
    labels=labels,
    percentage = percentages
))


tool_tips_i = [
    ("Share:" , "@percentage{0.0}%")
]

imports_pie_chart = figure(x_range=x_range,
                         y_range=y_range,
                         tooltips=tool_tips_i,
                         title=f"{country}-{partner}-Imports-{year}")


i = imports_pie_chart.annular_wedge(x=0, y=0, inner_radius=3.2, outer_radius=4.5,
                     start_angle="start", end_angle="end",
                     line_color="white", line_width=3, fill_color="colors" , 
                     source=imports_source,
                     legend_group="labels")

imports_pie_chart.legend.location = "center"

legend = Legend(location="center")


show(imports_pie_chart)


/var/folders/c3/rch9_l9x6pbg6mtk7lg7976r0000gp/T/ipykernel_6016/735057921.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/c3/rch9_l9x6pbg6mtk7lg7976r0000gp/T/ipykernel_6016/735057921.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
# top exporters/importers of a subgroup i.e. 2 digit SITC code

year = 2021
code = "51"

exporters = all_digit
exporters["export_percentage"] = (100*all_digit.groupby(by = ["location_code" ,"year" , "sitc_product_code_2"])["export_value"].transform("sum")) / (all_digit.groupby(by = ["year" , "sitc_product_code_2"])["export_value"].transform("sum"))
exporters = exporters.groupby(by = ["location_code" , "year" , "sitc_product_code_2"]).aggregate({"export_value":"sum" , "import_value":"sum" , "export_percentage":"max"})
exporters.reset_index(level=[0,1,2] , drop=False , inplace=True)
t = exporters[(exporters["year"]==year) * (exporters["sitc_product_code_2"] == code)].sort_values(by="export_percentage" , ascending=False).head(20)
t.head(10)

,location_code,year,sitc_product_code_2,export_value,import_value,export_percentage
157389,CHN,2021,51,7.464431e+10,5.609802e+10,15.905580
836381,USA,2021,51,4.367938e+10,5.355516e+10,9.307419
220514,DEU,2021,51,3.485394e+10,3.778859e+10,7.426851
390525,IRL,2021,51,3.327448e+10,8.462243e+09,7.090291
148691,CHE,2021,51,2.403943e+10,7.233371e+09,5.122440
450248,KOR,2021,51,2.366145e+10,1.565889e+10,5.041899
593944,NLD,2021,51,2.281646e+10,1.926082e+10,4.861845
382995,IND,2021,51,2.117006e+10,2.452350e+10,4.511021
425317,JPN,2021,51,1.949283e+10,1.512206e+10,4.153629
69000,BEL,2021,51,1.817762e+10,2.692047e+10,3.873377


In [44]:
# Composition of a given sector for country, partner, sector and year - (country , partner , sector , year)

country = "IND"
partner = "CHN"
sector = "5"
year = 2021

ind_china = country_partner[(country_partner["partner_code"] == "CHN")]

sector_name = (product_codes[product_codes["Code"] == sector]["Description"]).iloc[0]
if(sector == "services"):
    subproducts = services
else:
    subproducts = [sector + str(i) for i in range(1,10)]

g3_db = ind_china[np.isin(ind_china["sitc_product_code_2"] , subproducts) * (ind_china["location_code"] == country)]

pi_db = g3_db.groupby(by=["year" , "sitc_product_code_2"]).aggregate({"export_value":"sum" , "import_value":"sum" , "sitc_product_code_1":"max"})
total_7 = ind_china[(ind_china["sitc_product_code_1"] == sector) * (ind_china["location_code"] == country)].groupby(by=["year" , "sitc_product_code_1"]).aggregate({"export_value":"sum" , "import_value":"sum"})
total_7.reset_index(level=["year" , "sitc_product_code_1"] , drop=False , inplace=True)
pi_db.reset_index(level=["year" , "sitc_product_code_2"] , drop=False , inplace=True)
pi_db = pd.merge(pi_db , total_7 , how="left" , on=["year" , "sitc_product_code_1"] , suffixes=["" , "_total"])

x_range = Range1d(-5,5)
y_range = Range1d(-5,5)

inferno = palettes.Category20[len(subproducts)-1]

exports_angles_data = (pi_db[pi_db["year"] == year][["sitc_product_code_2" , "export_value"]].groupby("sitc_product_code_2").sum())
exports_angles_data.sort_values(by="export_value" , ascending=False , inplace=True)
exports_angles_data.reset_index(0 , drop=False , inplace=True)
exports_angles_data["labels"] = exports_angles_data["sitc_product_code_2"].apply(get_product_description)
labels = list(exports_angles_data["labels"].apply(lambda x : x[:30]))


exports_angles_data["angles"] = (2*np.pi*(exports_angles_data["export_value"]/exports_angles_data["export_value"].sum())).cumsum()
exports_angles_data["percentage"] = (100*exports_angles_data["export_value"])/exports_angles_data["export_value"].sum()
exports_start = [0] + list(exports_angles_data["angles"][:-1])
exports_end = list(exports_angles_data["angles"])
exports_percentages = list(exports_angles_data["percentage"])
sectors = exports_angles_data.index.get_level_values(0)


imports_angles_data = (pi_db[pi_db["year"] == year][["sitc_product_code_2" , "import_value" , "export_value"]].groupby("sitc_product_code_2").sum())
imports_angles_data.sort_values(by="export_value" , ascending=False , inplace=True)
subproducts_7 = list(imports_angles_data.index.get_level_values(0))


imports_angles_data["angles"] = (2*np.pi*(imports_angles_data["import_value"]/(imports_angles_data["import_value"].sum()))).cumsum()
imports_angles_data["percentage"] = (100*imports_angles_data["import_value"])/imports_angles_data["import_value"].sum()
imports_start = [0] + list(imports_angles_data["angles"][:-1])
imports_end = list(imports_angles_data["angles"])
imports_percentages = list(imports_angles_data["percentage"])


colours = inferno

exports_distribution_source = ColumnDataSource(dict(
    start  = exports_start,
    end    = exports_end,
    colors = colours,
    sectors = sectors,
    labels=labels,
    percentage = exports_percentages
))


imports_distribution_source = ColumnDataSource(dict(
    start  = imports_start,
    end    = imports_end,
    colors = colours,
    sectors = sectors,
    labels = labels,
    percentage = imports_percentages
))

tool_tips_e = [
    ("Share:" , "@percentage{0.0}%")
]

tool_tips_i = [
    ("Share:" , "@percentage{0.0}%")
]

exports_pie_chart = figure(x_range=x_range, 
                         y_range=y_range,
                         tooltips= tool_tips_e,
                         title=f"{country}-{partner}-{sector_name}-Exports {year}")

imports_pie_chart = figure(x_range=x_range,
                         y_range=y_range,
                         tooltips=tool_tips_i,
                         title=f"{country}-{partner}-{sector_name}-Imports {year}")

pie_row = layouts.row(children=[exports_pie_chart , imports_pie_chart] , sizing_mode="scale_both")

e = exports_pie_chart.annular_wedge(x=0, y=0, inner_radius=3.2, outer_radius=4.5,
                     start_angle="start", end_angle="end",
                     line_color="white", line_width=3, fill_color="colors" , 
                     source=exports_distribution_source,
                     legend_group="labels")

i = imports_pie_chart.annular_wedge(x=0, y=0, inner_radius=3.2, outer_radius=4.5,
                     start_angle="start", end_angle="end",
                     line_color="white", line_width=3, fill_color="colors" , 
                     source=imports_distribution_source,
                     legend_group="labels")

exports_pie_chart.legend.location = "center"
imports_pie_chart.legend.location = "center"

legend = Legend(location="center")


show(pie_row)


In [48]:
# GDP growth rate comparions for a list of countries in a year (countries[] , year)

gdp_growths = gdp_df.groupby(["location_code" , "year"]).sum()["gdp"].pct_change(periods=1)
year = 2021

gdp_comparisons = {"IND":"#405D5D" , "CHN":"#CC3333", "USA":"#708090" , "FRA":"#B19CD9" , "JPN":"#BA9CA6" , "AUS":"#78866B"}
values = []

for country in gdp_comparisons.keys():
    values.append(100*gdp_growths.loc[country , year])

data_source = ColumnDataSource(data=dict(
    x = list(gdp_comparisons.keys()),
    y = values,
    colour = list(gdp_comparisons.values())
))

tooltips = [
    ("growth: " , "@y%")
]

gdp_growth = figure(x_range = list(gdp_comparisons.keys()) , title = f"GDP growth rates for {year} ",
                   toolbar_location="below" , x_axis_label = "Countries" , y_axis_label="Percentage" , tooltips=tooltips)

gdp_growth.vbar(x = "x" , top = "y" , source=data_source , width = 0.5 , color="colour")
gdp_growth.xaxis.major_label_orientation = 1.2

show(gdp_growth)